In [1]:
# %pip install -r requirements.txt

In [2]:
# %pip install transformers

In [3]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
from collections import Counter
import os

d:\miniconda3\envs\torch-cuda-2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Annotaions

In [4]:
# Load the annotations
annotations_train_path = 'data/processed_annotations_train.csv'
annotations_train = pd.read_csv(annotations_train_path)

annotations_val_path = 'data/processed_annotations_valid.csv'
annotations_val = pd.read_csv(annotations_val_path)

In [5]:
print(annotations_train.head())

                                       attachment_id text  \
0  data/tensors\train_9ede40ec-0c67-4a57-8337-003...    Н   
1  data/tensors\train_8fb4e5c9-a7df-4bad-b7d7-9d8...    Н   
2  data/tensors\train_d228da91-495a-4d6d-833a-64b...    Н   
3  data/tensors\train_397a486d-967f-444b-afd8-de1...    Н   
4  data/tensors\train_e1200d57-9b53-4ffa-ab7d-e3e...    Н   

                            user_id  height  width  length  train  
0  185bd3a81d9d618518d10abebf0d17a8    1920   1080    95.0   True  
1  46dd04a1caa75ed3082b573cb5a3ad26    1920   1080    37.0   True  
2  db573f94204e56e0cf3fc2ea000e5bdc    1280    720    45.0   True  
3  0211b488644476dd0fec656ccb9b74fc    1920   1080    52.0   True  
4  b07a773bcb10b4f14f33d2b0e8ec58ba     720   1280    69.0   True  


In [6]:
print(annotations_val.head())

                                       attachment_id text  \
0  data/tensors\valid_d1131f18-57d2-412f-8261-aca...    Н   
1  data/tensors\valid_016df4ed-6e78-40a1-974f-e5d...    Н   
2  data/tensors\valid_45557bda-2f3a-4882-a430-9cf...    Н   
3  data/tensors\valid_ab6ef8b7-1971-4218-ad0f-bfe...    Н   
4  data/tensors\valid_49d3d3c3-7288-4a05-96bf-76d...    Н   

                            user_id  height  width  length  train  
0  0ab4f8e463cdded2e59d6001f4e1b487    1080   1920    42.0  False  
1  1860c7f3605089bf0eff0c4e72c7add4    1440   1440    41.0  False  
2  7c989fdb633be7453228f7c650097205    1920   1080    30.0  False  
3  ba230d5b794c9a7350356eddd3e9929c    1920    960    41.0  False  
4  3dd2ce2659aada17b976390004ebe322    1920    886    39.0  False  


### Pretrained models

In [7]:
# # Load Swin Transformer and MViTv2 models as generic models
swin_transformer_model = torch.jit.load('models/swin16-3.pt')
mvitv2_model = torch.jit.load('models/mvit16-4.pt')

In [8]:
# Make sure to call eval() if you're using the models for inference
swin_transformer_model.eval()
mvitv2_model.eval()

RecursiveScriptModule(
  original_name=Recognizer3D
  (data_preprocessor): RecursiveScriptModule(original_name=ActionDataPreprocessor)
  (backbone): RecursiveScriptModule(
    original_name=MViT
    (patch_embed): RecursiveScriptModule(
      original_name=PatchEmbed3D
      (projection): RecursiveScriptModule(original_name=Conv3d)
    )
    (blocks): RecursiveScriptModule(
      original_name=ModuleList
      (0): RecursiveScriptModule(
        original_name=MultiScaleBlock
        (norm1): RecursiveScriptModule(original_name=LayerNorm)
        (attn): RecursiveScriptModule(
          original_name=MultiScaleAttention
          (qkv): RecursiveScriptModule(original_name=Linear)
          (proj): RecursiveScriptModule(original_name=Linear)
          (pool_q): RecursiveScriptModule(original_name=Conv3d)
          (norm_q): RecursiveScriptModule(original_name=LayerNorm)
          (pool_k): RecursiveScriptModule(original_name=Conv3d)
          (norm_k): RecursiveScriptModule(original_name

In [9]:
additional_models = {
    # 'swin_transformer': swin_transformer_model,
    'mvitv2': mvitv2_model
}

In [10]:
def adjust_inputs_for_model(inputs):
    adjusted_inputs = inputs.squeeze()
    return adjusted_inputs

In [11]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [12]:
class PaddedSignLanguageDataset(Dataset):
    def __init__(self, annotations, transform=None, max_length=None):
        """
        Corrected custom dataset for loading sign language video tensors with padding.

        Args:
        annotations (DataFrame): DataFrame containing the annotations.
        transform (callable, optional): Optional transform to be applied on a sample.
        max_length (int, optional): Maximum length of the video tensors. If not provided, it will be calculated.
        """
        self.annotations = annotations
        self.transform = transform
        self.max_length = 132

        if self.max_length is None:
            # Calculate the maximum length among all tensors
            self.max_length = max(len(torch.load(row['attachment_id'], map_location=torch.device('cpu'))) for _, row in annotations.iterrows())

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        tensor_path = self.annotations.iloc[idx]['attachment_id']
        label = self.annotations.iloc[idx]['text']
        
        # Load the tensor
        tensor = torch.load(tensor_path, map_location=torch.device('cpu'))

        # Check if the tensor is empty or None
        if tensor is None or len(tensor) == 0:
            print(f"Empty tensor found at index {idx}.")
            return None, label


        # Pad the tensor to the maximum length
        padded_tensor = torch.zeros((self.max_length, *tensor[0].shape))
        padded_tensor[:len(tensor)] = torch.stack(tensor)
                
        # Apply transform if any
        if self.transform:
            padded_tensor = self.transform(padded_tensor)

        return padded_tensor, label

In [13]:
# Create the padded dataset and dataloader
padded_dataset_train = PaddedSignLanguageDataset(annotations_train)
padded_dataloader_train = DataLoader(padded_dataset_train, batch_size=4, shuffle=True)

padded_dataset_val = PaddedSignLanguageDataset(annotations_val)
padded_dataloader_val = DataLoader(padded_dataset_val, batch_size=4, shuffle=True)

### Labels Mapping

In [14]:
# Initialize an empty set to collect unique labels
unique_labels_train = set()

# Iterate over your dataset to collect unique labels
for _, label_data in padded_dataloader_train:
    unique_labels_train.update(label_data)

# Sort the labels for consistency
sorted_labels = sorted(unique_labels_train)

# Create the label mapping
train_label_mapping = {label: idx for idx, label in enumerate(sorted_labels)}

# Print the label mapping
print("Label Mapping:", train_label_mapping)

Label Mapping: {'Н': 0, 'вероятно': 1, 'значить': 2, 'обычный': 3, 'овца': 4, 'продлевать': 5, 'суматоха': 6, 'такой': 7, 'цветовой оттенок': 8, 'черный': 9}


In [15]:
# Initialize an empty set to collect unique labels
unique_labels_test = set()

# Iterate over your dataset to collect unique labels
for _, label_data in padded_dataloader_val:
    unique_labels_test.update(label_data)

# Sort the labels for consistency
sorted_labels = sorted(unique_labels_test)

# Create the label mapping
val_label_mapping = {label: idx for idx, label in enumerate(sorted_labels)}

# Print the label mapping
print("Label Mapping:", val_label_mapping)

Label Mapping: {'Н': 0, 'вероятно': 1, 'значить': 2, 'обычный': 3, 'овца': 4, 'продлевать': 5, 'суматоха': 6, 'такой': 7, 'цветовой оттенок': 8, 'черный': 9}


In [16]:
from classes import classes

In [17]:
results = {}

In [18]:
class TorchScriptModelWrapper(nn.Module):
    def __init__(self, scripted_model, output_classes=10):
        super(TorchScriptModelWrapper, self).__init__()
        self.scripted_model = scripted_model
        # Assuming you know the number of features from the original model's output
        self.new_fc = nn.Linear(1001, output_classes)

    def forward(self, x):
        x = self.scripted_model(x)
        # The output of the scripted model is then passed through the new fully connected layer
        x = self.new_fc(x)
        return x

In [19]:
checkpoint_dir = 'checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

In [20]:
for model_name, model in additional_models.items():
    if torch.jit.isinstance(model, torch.jit.ScriptModule):
        additional_models[model_name] = TorchScriptModelWrapper(model)

    model = additional_models[model_name]
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    loss_function = nn.CrossEntropyLoss()
    model.eval()

    val_correct_predictions = 0
    val_total_predictions = 0

    for epoch in range(10):
        with tqdm(enumerate(padded_dataloader_val), total=len(padded_dataloader_val), desc=f"Model: {model_name}") as t_outer:
            for i, data in t_outer:
                inputs, label_data = data
                if len(inputs) == 0 or len(label_data) == 0:
                    continue

                labels = torch.tensor([val_label_mapping[label] for label in label_data], dtype=torch.long)

                batch_corr = 0.
                batch_total = 0.

                with tqdm(enumerate(inputs), total=len(inputs), leave=False, desc="Processing inputs") as t_inner:
                    for e, inp in t_inner:
                        target = labels[e]
                        target_probs = torch.zeros(1, 10)
                        target_probs[0][target] = 1
                        j = 0
                        inp_pred = []
                        num_splits = inp.shape[0] // 16
                        while j + num_splits <= inp.shape[0]:
                            input_splits = inp[j:j+num_splits]
                            input_splits = torch.reshape(input_splits, (1, 3, -1, 64, 64))
                            input_splits = np.stack(input_splits[:num_splits], axis=1)[None][None]
                            input_splits = torch.from_numpy(input_splits)
                            
                            optimizer.zero_grad()

                            outputs = model(input_splits)

                            # print(outputs.shape, target_probs.shape)
                            loss = loss_function(outputs, target_probs)
                            loss.backward()

                            optimizer.step()

                            pred_class = torch.argmax(outputs, dim=1)

                            inp_pred.append(pred_class.item())
                            j += 1

                        pred_counter = Counter(inp_pred)
                        pred_label = pred_counter.most_common(1)[0][0]

                        print(f'pred_label {pred_label}, label {target}')

                        batch_total += 1
                        batch_corr += (pred_label == labels[e]).sum().item()

                        t_inner.set_postfix(batch_accuracy=batch_corr/batch_total)

                val_correct_predictions += batch_corr
                val_total_predictions += batch_total

            val_accuracy = val_correct_predictions / val_total_predictions
            t_outer.set_postfix(model_accuracy=val_accuracy, epoch=epoch)

            checkpoint_path = os.path.join(checkpoint_dir, f"{model_name}_epoch_{epoch}.pt")
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
                'val_accuracy': val_accuracy
            }, checkpoint_path)


            results[model_name] = val_accuracy

Model: mvitv2:   0%|          | 0/13 [00:00<?, ?it/s]

pred_label 7, label 7


pred_label 7, label 6


pred_label 7, label 7


Model: mvitv2:   8%|▊         | 1/13 [13:05<2:37:03, 785.26s/it]

...

pred_label 9, label 3


pred_label 3, label 5


Model: mvitv2:  85%|████████▍ | 11/13 [2:50:41<33:12, 996.26s/it]

pred_label 5, label 6


pred_label 5, label 5
